Classification!


In [1]:
from google.colab import drive

drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
Â·Â·Â·Â·Â·Â·Â·Â·Â·Â·
Mounted at /content/drive


In [2]:
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

import numpy

# dimensions of our images.
img_width, img_height = 150, 150


if K.image_data_format() == "channels_first":
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)


model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation("sigmoid"))
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop", metrics=["accuracy"])

# from keras.models import load_model
model.load_weights("./weights.h5")

In [11]:
myPic = "./data/live/marley2.jpg"
test_image = image.load_img(myPic, target_size=(img_width, img_height))
test_image = image.img_to_array(test_image)
test_image = test_image.reshape(input_shape)
test_image = numpy.expand_dims(test_image, axis=0)
result = model.predict(test_image, verbose=0)
print(result[0][0])

myPic2 = "./data/live/calevra1.jpg"
test_image2 = image.load_img(myPic2, target_size=(img_width, img_height))
test_image2 = image.img_to_array(test_image2)
test_image2 = test_image2.reshape(input_shape)
test_image2 = numpy.expand_dims(test_image2, axis=0)
result = model.predict(test_image2, verbose=0)
print(result[0][0])

0.0
1.0


In [9]:
myPic = "./data/live/corgi3.jpg"
test_image = image.load_img(myPic, target_size=(img_width, img_height))
test_image = image.img_to_array(test_image)
test_image = test_image.reshape(input_shape)
test_image = numpy.expand_dims(test_image, axis=0)
result = model.predict(test_image, verbose=0)
print(result[0][0])

myPic2 = "./data/live/cat8.jpg"
test_image2 = image.load_img(myPic2, target_size=(img_width, img_height))
test_image2 = image.img_to_array(test_image2)
test_image2 = test_image2.reshape(input_shape)
test_image2 = numpy.expand_dims(test_image2, axis=0)
result = model.predict(test_image2, verbose=0)
print(result[0][0])

3.7968094e-20
1.0


In [17]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3947215355304067952
xla_global_id: -1
]


2023-12-01 19:29:51.740616: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-01 19:29:51.740682: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [6]:
"""
In this example we iterating over 4 search queries,
doing pagination on each query until results is present,
and extracting original size image + optionally saving locally
"""

import urllib.request
from serpapi import GoogleSearch


def serpapi_get_google_images():
    image_results = []

    for query in ["racoon"]:
        # search query parameters
        params = {
            # search engine. Google, Bing, Yahoo, Naver, Baidu...
            "engine": "google",
            "q": query,  # search query
            "tbm": "isch",  # image results
            "num": "100",  # number of images per page
            # page number: 0 -> first page, 1 -> second...
            "ijn": 0,
            # https://serpapi.com/manage-api-key
            "api_key": "a743de5d2b5b31d044cd18b3c2807358f4ac1e537c2545266b58eae15b8c8f1f",
            # other query parameters: hl (lang), gl (country), etc
        }

        search = GoogleSearch(params)  # where data extraction happens

        images_is_present = True
        while images_is_present:
            results = search.get_dict()  # JSON -> Python dictionary

            # checks for "Google hasn't returned any results for this query."
            if "error" not in results:
                for image in results["images_results"]:
                    if image["original"] not in image_results:
                        image_results.append(image["original"])

                # update to the next page
                params["ijn"] += 1
            else:
                print(results["error"])
                images_is_present = False

    # -----------------------
    # Downloading images

    for index, image in enumerate(results["images_results"], start=1):
        print(f"Downloading {index} image...")

        opener = urllib.request.build_opener()
        opener.addheaders = [
            (
                "User-Agent",
                "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36",
            )
        ]
        urllib.request.install_opener(opener)

        urllib.request.urlretrieve(
            image["original"], f"./data/train/racoons/{index}.jpg"
        )

    print(json.dumps(image_results, indent=2))
    print(len(image_results))

In [5]:
serpapi_get_google_images()

Google Images hasn't returned any results for this query.


KeyError: 'images_results'